In [1]:
# IMPORTANDO AS BIBLIOTECAS

import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import requests  # obter dados do Bacen, usando a API pública direto do Bacen
from datetime import datetime  # Importação correta

In [ ]:
## Dar cor para os gráficos: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
from pylab import mpl, plt # importando o pacote

plt.style.use('seaborn') # mudando a cor da paleta dos gráficos
mpl.rcParams['font.family'] = 'serif' # mudando a fonte dos gráficos

<ipython-input-3-81bbd8727fb6>:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn') # mudando a cor da paleta dos gráficos


In [ ]:
# Função para baixar dados do Yahoo Finance
def get_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)
    return data['Adj Close']

In [ ]:
# Função para obter dados do Bacen
def get_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)
    return data['Adj Close']

# Função para obter dados do Banco Central do Brasil
def get_bcb_data(series_id, start, end):
    # Converter datas para o formato dd/MM/yyyy
    start = datetime.strptime(start, '%Y-%m-%d').strftime('%d/%m/%Y')
    end = datetime.strptime(end, '%Y-%m-%d').strftime('%d/%m/%Y')

    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_id}/dados?formato=json&dataInicial={start}&dataFinal={end}'
    response = requests.get(url)
    data = response.json()

    # Depuração: verificar o tipo de dados recebidos
    if isinstance(data, list) and len(data) > 0:
        print(f"Dados recebidos para series_id {series_id}: {data[:5]}...")  # Exibe apenas os primeiros 5 registros para brevidade
    else:
        print(f"Formato inesperado de dados recebidos para series_id {series_id}: {data}")
        raise ValueError(f"Unexpected data format received from BCB for series_id {series_id}")

    df = pd.DataFrame(data)

    if 'data' in df.columns and 'valor' in df.columns:
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df.set_index('data', inplace=True)
        df = df.rename(columns={'valor': series_id})
        df = df.astype(float)
        return df
    else:
        raise ValueError(f"Unexpected data format received from BCB for series_id {series_id}")

In [ ]:
# Configurações de data
start_date = '2020-01-01'
end_date = '2023-01-01'

In [ ]:
# Baixar dados de preços ajustados para os ativos (DEFININDO A CARTEIRA)

tickers = ['AAPL', 'MSFT', 'GOOGL']  # Apple, Microsoft e Google
asset_prices = get_data(tickers, start_date, end_date)
# tickers = ['AMAR3.SA', 'UNIP6.SA', 'JBSS3.SA', 'EMBR3.SA', 'VALE3.SA', 'BBAS3.SA', 'BOVA11.SA', 'CSNA3.SA', 'WEGE3.SA', 'USIM5.SA']

[*********************100%%**********************]  3 of 3 completed


In [ ]:
# Obter dados reais do Brasil

# No modelo APT vamos usar 3 fatores: Selic (taxa de juros), PIB e Inflação (IPCA)

# Código SGS para Selic: 432
# Código SGS para PIB real trimestral: 7326
# Código SGS para IPCA: 433
selic = get_bcb_data(432, start_date, end_date)
gdp = get_bcb_data(7326, start_date, end_date)
ipca = get_bcb_data(433, start_date, end_date)

Dados recebidos para series_id 432: [{'data': '01/01/2020', 'valor': '4.50'}, {'data': '02/01/2020', 'valor': '4.50'}, {'data': '03/01/2020', 'valor': '4.50'}, {'data': '04/01/2020', 'valor': '4.50'}, {'data': '05/01/2020', 'valor': '4.50'}]...
Dados recebidos para series_id 7326: [{'data': '01/01/2020', 'valor': '-3.28'}, {'data': '01/01/2021', 'valor': '4.76'}, {'data': '01/01/2022', 'valor': '3.02'}, {'data': '01/01/2023', 'valor': '2.91'}]...
Dados recebidos para series_id 433: [{'data': '01/01/2020', 'valor': '0.21'}, {'data': '01/02/2020', 'valor': '0.25'}, {'data': '01/03/2020', 'valor': '0.07'}, {'data': '01/04/2020', 'valor': '-0.31'}, {'data': '01/05/2020', 'valor': '-0.38'}]...


In [ ]:
# Ajustar os dados do PIB para uma base mensal
gdp_monthly = gdp.resample('M').ffill()

In [ ]:
# Calcular a taxa de crescimento do PIB mensal
gdp_monthly['gdp_growth'] = gdp_monthly['7326'].pct_change()

KeyError: '7326'

In [ ]:
# Calcular a inflação mensal
ipca_monthly = ipca.resample('M').ffill()
ipca_monthly['inflation'] = ipca_monthly['433'].pct_change()

KeyError: '433'

In [ ]:
# Combinar os fatores em um único DataFrame
factors_df = pd.DataFrame({
    'selic': selic['432'],
    'gdp_growth': gdp_monthly['gdp_growth'],
    'inflation': ipca_monthly['inflation']
}).dropna()

In [ ]:
# Calcular os retornos diários
asset_returns = asset_prices.pct_change().dropna()

In [ ]:
# Sincronizar as datas
factors_df = factors_df.reindex(asset_returns.index, method='ffill')

In [ ]:
# Regressão APT
X = factors_df.values
X = sm.add_constant(X)  # Adicionar constante para o intercepto
results = {}

for ticker in tickers:
    y = asset_returns[ticker].values
    model = sm.OLS(y, X).fit()
    results[ticker] = model.summary()

In [ ]:
# Exibir resultados
for ticker in tickers:
    print(f'Results for {ticker}:')
    print(results[ticker])
    print('\n')